In [ ]:
from google.colab import drive
drive.mount('/content/drive')



MessageError: Error: credential propagation was unsuccessful

In [1]:
import pandas as pd

# Try reading the CSV file with 'ISO-8859-1' encoding
data = pd.read_csv('/content/drive/MyDrive/open iit data 24/Dataset_US/combined_data1.csv', encoding='ISO-8859-1')
data.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shared with me/open iit data 24/Dataset_US/combined_data1.csv'

In [ ]:
!pip install geopy


In [ ]:
!pip install opencage


In [ ]:
import time
import pandas as pd
from opencage.geocoder import OpenCageGeocode

# Replace 'YOUR_API_KEY' with your actual OpenCage API key
api_key = 'c4a1893e4a2441e48e50a36114eacda3'
geocoder = OpenCageGeocode(api_key)

df = pd.DataFrame(data)

# Function to get latitude and longitude with OpenCage API
def get_lat_lon_opencage(city_name):
    try:
        results = geocoder.geocode(city_name)
        if results:
            return results[0]['geometry']['lat'], results[0]['geometry']['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching location for {city_name}: {e}")
        return None, None

# Apply the function to the first 20 entries of the OriginCityName column with delay
lat_lon_data = []
for city in df['OriginCityName']:  # Fetching only the first 20 entries
    lat_lon_data.append(get_lat_lon_opencage(city))
    time.sleep(1)  # 1-second delay between requests

# Add Latitude and Longitude to DataFrame
df[['Latitude', 'Longitude']] = pd.DataFrame(lat_lon_data, columns=['Latitude', 'Longitude'])

# Display the updated DataFrame for the first 20 entries
df.head(20)


In [ ]:
import time
import pandas as pd
from opencage.geocoder import OpenCageGeocode

# Replace 'YOUR_API_KEY' with your actual OpenCage API key
api_key = 'c4a1893e4a2441e48e50a36114eacda3'
geocoder = OpenCageGeocode(api_key)

df = pd.DataFrame(data)

# Function to get latitude and longitude with OpenCage API
def get_lat_lon_opencage(city_name):
    try:
        results = geocoder.geocode(city_name)
        if results:
            return results[0]['geometry']['lat'], results[0]['geometry']['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching location for {city_name}: {e}")
        return None, None

# Get unique city names
unique_cities = df['OriginCityName'].unique()

# Create a dictionary to store lat/lon values
lat_lon_dict = {}

# Fetch lat/lon for unique cities
for city in unique_cities:
    lat_lon_dict[city] = get_lat_lon_opencage(city)
    time.sleep(1)  # 1-second delay between requests

# Replace the latitude and longitude in the original DataFrame
df['Latitude'] = df['OriginCityName'].map(lambda x: lat_lon_dict[x][0] if lat_lon_dict[x] else None)
df['Longitude'] = df['OriginCityName'].map(lambda x: lat_lon_dict[x][1] if lat_lon_dict[x] else None)

# Display the updated DataFrame
df.head()


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('output_file.csv', index=False)


In [ ]:
from google.colab import files
# Download the CSV file
files.download('output_file.csv')

In [ ]:
df['Latitude']

In [ ]:
df['Longitude']

In [ ]:
# Convert 'FlightDate' from MM/DD/YYYY to datetime
df['FlightDate'] = pd.to_datetime(df['FlightDate'], format='%m/%d/%Y', errors='coerce')

# Print the DataFrame after conversion
print("\nDataFrame after converting FlightDate:")
df.head()

# Optionally drop rows with missing FlightDate
df = df.dropna(subset=['FlightDate'])

# Print the final DataFrame
print("\nFinal DataFrame after dropping missing FlightDate:")
df.head()

In [ ]:
!pip install meteostat


In [ ]:
import pandas as pd
from meteostat import Point, Daily

# Convert FlightDate to datetime
df['FlightDate'] = pd.to_datetime(df['FlightDate'])

# Function to get weather data using Meteostat
def get_weather_data(latitude, longitude, flight_date):
    location = Point(latitude, longitude)

    # Set the start and end date to the flight date
    start_date = flight_date
    end_date = flight_date

    # Fetch daily weather data
    data = Daily(location, start=start_date, end=end_date)
    weather_data = data.fetch()

    return weather_data

# Collect weather data
weather_results = []

for index, row in df.head(20).iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    flight_date = row['FlightDate']

    # Fetch weather data for the specified latitude, longitude, and flight date
    weather_data = get_weather_data(latitude, longitude, flight_date)

    # Append the weather data to the results list
    weather_results.append(weather_data)

# Convert the results into a single DataFrame for easier analysis
combined_weather_data = pd.concat(weather_results)

# Reset index if needed
combined_weather_data.reset_index(inplace=True)

# Display the combined weather data
print(combined_weather_data)

In [ ]:
# Convert FlightDate to datetime format if necessary
df['FlightDate'] = pd.to_datetime(df['FlightDate'])

# Filter rows where WeatherDelay is non-null
non_null_weather_delay = df[df['WeatherDelay'].notnull()]

# Display the resulting DataFrame
non_null_weather_delay

In [ ]:
# Find the unique combinations of OriginCityName and FlightDate
unique_combinations = non_null_weather_delay.groupby(['OriginCityName', 'FlightDate']).size()

# Get the number of unique combinations
unique_combinations_count = unique_combinations.count()

# Print the result
print("Number of unique combinations:", unique_combinations_count)

In [ ]:
import time
import pandas as pd
from opencage.geocoder import OpenCageGeocode

# Replace 'YOUR_API_KEY' with your actual OpenCage API key
api_key = 'c4a1893e4a2441e48e50a36114eacda3'
geocoder = OpenCageGeocode(api_key)

df = pd.DataFrame(data)

# Function to get latitude and longitude with OpenCage API
def get_lat_lon_opencage(city_name):
    try:
        results = geocoder.geocode(city_name)
        if results:
            return results[0]['geometry']['lat'], results[0]['geometry']['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching location for {city_name}: {e}")
        return None, None

# Apply the function to the OriginCityName column with delay
lat_lon_data = []
for city in df['OriginCityName']:
    lat_lon_data.append(get_lat_lon_opencage(city))
    time.sleep(1)  # 1-second delay between requests

# Add Latitude and Longitude to DataFrame
df[['Latitude', 'Longitude']] = pd.DataFrame(lat_lon_data, columns=['Latitude', 'Longitude'])

# Display the updated DataFrame
print(df.head())


In [ ]:
import pandas as pd
from datetime import datetime

# Sample dataset with possible NaN values in DepTime

df = pd.DataFrame(data)

# Remove rows where DepTime or FlightDate is NaN
df = df.dropna(subset=['DepTime', 'FlightDate'])

# Function to normalize DepTime
def normalize_deptime(deptime):
    deptime_str = str(int(float(deptime)))  # Convert to int first to remove decimal points
    deptime_str = deptime_str.zfill(4)  # Pad with leading zeros if necessary
    hour = int(deptime_str[:2])  # Extract hours
    minute = int(deptime_str[2:])  # Extract minutes

    # Adjust hour if it is 24
    if hour == 24:
        hour = 0

    return f'{hour:02d}:{minute:02d}:00'  # Return in HH:MM:SS format

# Apply normalization to DepTime
df['NormalizedDepTime'] = df['DepTime'].apply(normalize_deptime)

# Combine FlightDate and NormalizedDepTime to create full datetime
df['FullDepTime'] = df['FlightDate'] + ' ' + df['NormalizedDepTime']

# Convert to datetime object
df['FullDepTime'] = pd.to_datetime(df['FullDepTime'], format='%Y-%m-%d %H:%M:%S')

# Convert FullDepTime to UNIX timestamp (for use in weather API)
df['DepTimestamp'] = df['FullDepTime'].apply(lambda x: int(x.timestamp()))

df.head()


In [ ]:
# Function to get city coordinates (latitude, longitude)
def get_city_coordinates(city_name, api_key):
    url = f'https://api.opencagedata.com/geocode/v1/json?q={city_name}&key={api_key}'
    response = requests.get(url)

    # Debug: Print the response status and URL
    print(f'Fetching coordinates for: {city_name}, URL: {url}, Status Code: {response.status_code}')

    if response.status_code == 200:
        data = response.json()

        # Debug: Print the full API response
        print(f'API response for {city_name}: {data}')

        if data['results']:
            return data['results'][0]['geometry']['lat'], data['results'][0]['geometry']['lng']

    return None, None


In [ ]:
import pandas as pd
import requests

# NOAA CDO API Token
token = 'xCwfPAyluffdosbcCNOqMylHyDYILTjo'

# OpenCage API Key
opencage_api_key = 'c4a1893e4a2441e48e50a36114eacda3'  # Replace with your actual OpenCage API key

# Function to get city coordinates using OpenCage Geocoding API
def get_city_coordinates(city_name, opencage_api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json"
    params = {
        'q': city_name,
        'key': opencage_api_key,
        'limit': 1  # Limit to one result
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        results = response.json()['results']
        if results:
            lat = results[0]['geometry']['lat']
            lon = results[0]['geometry']['lng']
            return lat, lon
        else:
            return None, None
    elif response.status_code == 401:
        print(f"Unauthorized access. Check your API key for {city_name}.")
        return None, None
    else:
        print(f"Error fetching coordinates for {city_name}: {response.status_code}")
        return None, None

# Function to get NOAA stations based on city coordinates
def get_nearby_noaa_stations(lat, lon, token):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations"
    params = {
        'datasetid': 'GHCND',  # Global Historical Climatology Network (Daily)
        'limit': 5,            # Limit the number of stations to retrieve
        'radius': 50,          # Search within 50 km radius
        'units': 'metric',
        'lat': lat,
        'lon': lon
    }
    headers = {'token': token}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()  # Returns the list of stations
    except Exception as e:
        print(f"Error fetching NOAA stations for coordinates ({lat}, {lon}): {e}")
        return None

# Function to fetch weather data from NOAA for a given station on a specific date
def get_noaa_weather(station_id, date, token):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    params = {
        'datasetid': 'GHCND',       # Daily summaries dataset
        'locationid': station_id,    # NOAA Station ID
        'startdate': date,           # Start date in YYYY-MM-DD format
        'enddate': date,             # End date in YYYY-MM-DD format
        'datatypeid': 'TMAX',        # Example: Maximum temperature
        'limit': 1000                # Limit the number of results
    }
    headers = {'token': token}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()  # Returns the weather data
    except Exception as e:
        print(f"Error fetching weather data for station {station_id} on {date}: {e}")
        return None

# Load your dataset (assuming 'data' is already defined)
# Example: df = pd.DataFrame(data)

# Example usage for your dataset
city_to_noaa_id = {}  # Dictionary to store the mapping of cities to station IDs

for index, row in df.head(20).iterrows():
    city_name = row['OriginCityName']  # Extract city name from dataset

    # Fetch city coordinates using OpenCage
    lat, lon = get_city_coordinates(city_name, opencage_api_key)

    if lat is None or lon is None:
        print(f"Could not find coordinates for {city_name}")
        continue

    # Get nearby NOAA stations for the city's coordinates
    noaa_stations = get_nearby_noaa_stations(lat, lon, token)

    if noaa_stations and 'results' in noaa_stations:
        # Assuming you take the first station as the closest one
        station_id = noaa_stations['results'][0]['id']
        city_to_noaa_id[city_name] = station_id  # Map city to station ID
        print(f"Mapped {city_name} to NOAA Station ID: {station_id}")

        # Fetch weather data for the day of the flight departure
        dep_date = row['FlightDate']#.strftime('%Y-%m-%d')  # Convert to YYYY-MM-DD
        weather_data = get_noaa_weather(station_id, dep_date, token)

        # Extract and store relevant weather data
        if weather_data and 'results' in weather_data:
            for result in weather_data['results']:
                if result['datatype'] == 'TMAX':  # Maximum temperature
                    df.loc[index, 'max_temp'] = result['value']
                    print(f"Max temperature for {city_name} on {dep_date}: {result['value']}")
                elif result['datatype'] == 'TMIN':  # Minimum temperature
                    df.loc[index, 'min_temp'] = result['value']
                    print(f"Min temperature for {city_name} on {dep_date}: {result['value']}")
        else:
            print(f"No weather data found for {city_name} on {dep_date}")
    else:
        print(f"No NOAA stations found near {city_name}")

# After the loop, city_to_noaa_id will contain the mappings
city_to_noaa_id
# Display the updated DataFrame with weather data
df.head(20)


In [ ]:
opencage_api_key = 'c4a1893e4a2441e48e50a36114eacda3'

def check_api_key(api_key):
    # Test the API key with a sample query
    url = f"https://api.opencagedata.com/geocode/v1/json?q=London&key={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        print("API Key is valid.")
    elif response.status_code == 401:
        print("Unauthorized access. Check your API key.")
    else:
        print(f"Error: {response.status_code}")

# Check the OpenCage API Key
check_api_key(opencage_api_key)
